<a href="https://colab.research.google.com/github/2241031c/Night-Lights-Dissertation/blob/main/NL_6_7_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##This notebook will run a spatial regression for years 2018-2023 for Tomintoul. To run for a different area of interest just change the file path's

In [ ]:
#Install libraries
!pip install pysal

In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from google.colab import drive
from osgeo import gdal
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler
from pysal.model import spreg
from pysal.lib import weights
from libpysal.weights import Queen
import matplotlib.pyplot as plt
import rasterio
from rasterio.features import rasterize
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
import os
from spreg import ML_Lag, ML_Error
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define file paths - change these depending on where data is being pulled from
roads_shapefile = '/content/drive/MyDrive/NightLight/ArcGIS/digimap/aoi/Tomintoul_roads.shp'
buildings_shapefile = '/content/drive/MyDrive/NightLight/ArcGIS/digimap/aoi/Tomintoul_buildings.shp'
viirs_dir = '/content/drive/MyDrive/EarthEngineExports'
output_dir = '/content/drive/MyDrive/NightLight/ArcGIS/digimap/Numpy'

In [ ]:
#Define temporal range
years = range(2018, 2024)

# Load shapefiles
roads = gpd.read_file(roads_shapefile)
buildings = gpd.read_file(buildings_shapefile)

# Buffer distance
buffer_distance = 100  # meters

# Buffer the roads and buildings
buffered_roads = roads.buffer(buffer_distance)
buffered_buildings = buildings.buffer(buffer_distance)

# Define bounds and resolution for raster
resolution = 750  # VIIRS spatial resolution
bounds = buffered_roads.total_bounds

In [ ]:
# Create raster template
transform = from_bounds(*bounds, width=int((bounds[2] - bounds[0]) / resolution), height=int((bounds[3] - bounds[1]) / resolution))
out_shape = (int((bounds[3] - bounds[1]) / resolution), int((bounds[2] - bounds[0]) / resolution))

# Rasterize buffered roads and buildings
road_raster = rasterize([(geom, 1) for geom in buffered_roads.geometry], out_shape=out_shape, transform=transform, fill=0)
building_raster = rasterize([(geom, 1) for geom in buffered_buildings.geometry], out_shape=out_shape, transform=transform, fill=0)

# Save the rasterized data as numpy arrays
np.save(os.path.join(output_dir, 'road_influence.npy'), road_raster)
np.save(os.path.join(output_dir, 'building_influence.npy'), building_raster)

In [ ]:
# Convert TIFF to numpy arrays
def tiff_to_npy(tiff_path, npy_path):
    with rasterio.open(tiff_path) as src:
        array = src.read(1)
        np.save(npy_path, array)
    return array

# Convert all years
def convert_all_years(years, viirs_dir, output_dir):
    viirs_arrays = {}
    for year in years:
        tiff_path = os.path.join(viirs_dir, f'VIIRS_Tomintoul_{year}.tif')
        npy_path = os.path.join(output_dir, f'VIIRS_Tomintoul_{year}.npy')
        viirs_arrays[year] = tiff_to_npy(tiff_path, npy_path)
    return viirs_arrays

viirs_arrays = convert_all_years(years, viirs_dir, output_dir)

# Load and flatten numpy arrays
def load_and_flatten_npy(npy_path):
    array = np.load(npy_path)
    return array.flatten()

def load_and_flatten_tiff(tiff_path):
    with rasterio.open(tiff_path) as src:
        array = src.read(1)
    return array.flatten()

# Ensure data types are consistent and replace -9999 and NaN with 0
def clean_array(array):
    array = np.where(np.logical_or(array == -9999, np.isnan(array)), 0, array)
    return array.astype(np.float32)

In [ ]:
# Function to resample arrays to match the target array shape
def resample_array(src_array, src_transform, src_crs, target_transform, target_shape, resampling_method=Resampling.nearest):
    resampled_array = np.empty(target_shape, dtype=src_array.dtype)
    reproject(
        source=src_array,
        destination=resampled_array,
        src_transform=src_transform,
        src_crs=src_crs,
        dst_transform=target_transform,
        dst_crs=src_crs,
        resampling=resampling_method
    )
    return resampled_array

In [ ]:
# Automate the spatial regression for each year
for year in years:
    # Load VIIRS data for the year
    viirs_radiance = clean_array(load_and_flatten_npy(os.path.join(output_dir, f'VIIRS_Tomintoul_{year}.npy')))

    # Load and resample road and building rasters to match the VIIRS raster
    with rasterio.open(os.path.join(viirs_dir, f'VIIRS_Tomintoul_{year}.tif')) as viirs_src:
        viirs_transform = viirs_src.transform
        viirs_shape = viirs_src.shape
        viirs_crs = viirs_src.crs

    road_raster_resampled = resample_array(
        src_array=road_raster,
        src_transform=transform,
        src_crs='EPSG:4326',
        target_transform=viirs_transform,
        target_shape=viirs_shape
    )

    building_raster_resampled = resample_array(
        src_array=building_raster,
        src_transform=transform,
        src_crs='EPSG:4326',
        target_transform=viirs_transform,
        target_shape=viirs_shape
    )
    # Flatten and clean the resampled arrays
    road_raster_flat = clean_array(road_raster_resampled.flatten())
    building_raster_flat = clean_array(building_raster_resampled.flatten())

    # Create DataFrame
    data = pd.DataFrame({
        'radiance': viirs_radiance,
        'road_influence': road_raster_flat,
        'building_influence': building_raster_flat,
    })

    # Perform spatial regression
    n_obs = data.shape[0]
    w = weights.lat2W(n_obs, 1)
    model = ML_Lag(data[['radiance']].values, data[['road_influence', 'building_influence']].values, w=w)

    # Output results
    print(f"Year {year} Model Summary")
    print(model.summary)

    # Perform GWR
    #coords = np.column_stack([data.index % viirs_shape[1], data.index // viirs_shape[1]])  # Generate coordinates
    #sel_bw = Sel_BW(coords, data[['radiance']].values, data[['road_influence', 'building_influence']].values)
    #bw = sel_bw.search()
    #gwr_model = GWR(coords, data[['radiance']].values, data[['road_influence', 'building_influence']].values, bw).fit()
    #print(f"Year {year} GWR Model Summary")
    #print(gwr_model.summary())